In [49]:
import csv
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import *
from sklearn.svm import SVR
from sklearn import svm
from sklearn.metrics import mean_absolute_error,mean_squared_error
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

In [50]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

# Pre-processing the Data

In [51]:
def pre_process_data(data,null_threshold):
    data.drop(columns=['Unixtime','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 <= data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=0,inplace=True)
    return data

# Removing columns based on the dependent column

In [52]:
def dependent_column(data,column):
    cols = [col for col in data.columns if "next" not in col.lower()]
    cols.append(column)
    data = data[cols]
    return (data,column)

# ---

In [53]:
def Model(dataset,y):
    X = dataset.loc[:, dataset.columns != y]
    Y = dataset.loc[:, dataset.columns == y]
    count = 3
    step = 1000
    start = 1
    end = 10000
    while(count >= 0):
        print("start--------->"+str(start))
        print("end------------>"+str(end))
        print("step----------->"+str(step))
        best,step = bestParameters(start,end,step,X,Y)
        for key in best:
            if key == 'C':
                c = best[key]
                print(c)
        if(count == 3):
            start = c - 1000
            end = c + 1000
        if(count == 2):
            start = c - 100
            end = c + 100
        if(count == 1):
            start = c - 10
            end = c + 10
        step = step/10
        count = count - 1 
    return c

In [54]:
def bestParameters(start,end,step,X,Y):
    parameters = {'kernel': ['rbf'], 'C': np.arange(start,end,step),'gamma': [0.0001],'epsilon':[0.01]}
    print("parameters ", parameters)
    svr = svm.SVR(parameters['gamma'][0])
    clf = GridSearchCV(svr, parameters, scoring = 'neg_mean_squared_error')
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    clf.fit(X_train,y_train)
    best = clf.best_params_
    print("best ", best)
    return (best,step)

In [55]:
%%time
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if filename.startswith("mod") and filename.endswith("GR.csv"):
        print(filename)
        name = filename.split(".")[0]
        df = pd.read_csv(os.path.join(path,"Data\Stock\\" + filename))
        df = pre_process_data(df,60)
        dataframe, y = dependent_column(df, "Next Day Close Price GR")
        
        c = Model(dataframe,y)
        print(c)
        
        X = dataframe.loc[:, dataframe.columns != y]
        Y = dataframe.loc[:, dataframe.columns == y]
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
        svr_rbf = SVR(kernel='rbf', C = 144, gamma = 0.0001, epsilon = 0.01)
        y_rbf = svr_rbf.fit(X_train, y_train).predict(X_test)
        
        
        r = r2_score(y_test, y_rbf)
        mse = mean_squared_error(y_test, y_rbf)
        mae = mean_absolute_error(y_test, y_rbf)
        med = median_absolute_error(y_test, y_rbf)
        var = explained_variance_score(y_test, y_rbf)
        print("R square "+str(r))
        print('-------')
        print("mean sqauare error ----------->"+str(mse))
        print('-------')
        print("mean absolute error --------->"+str(mae))
        print('-------')
        print("median absolute error ------------>"+str(med))
        print('-------')
        print("explained variance score ------------>"+str(var))
        
        break

mod500112GR.csv
start--------->1
end------------>10000
step----------->1000
parameters  {'kernel': ['rbf'], 'C': array([   1, 1001, 2001, 3001, 4001, 5001, 6001, 7001, 8001, 9001]), 'gamma': [0.0001], 'epsilon': [0.01]}
best  {'C': 1, 'epsilon': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}
1
start--------->-999
end------------>1001
step----------->100.0
parameters  {'kernel': ['rbf'], 'C': array([-999., -899., -799., -699., -599., -499., -399., -299., -199.,
        -99.,    1.,  101.,  201.,  301.,  401.,  501.,  601.,  701.,
        801.,  901.]), 'gamma': [0.0001], 'epsilon': [0.01]}
best  {'C': 1.0, 'epsilon': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}
1.0
start--------->-99.0
end------------>101.0
step----------->10.0
parameters  {'kernel': ['rbf'], 'C': array([-99., -89., -79., -69., -59., -49., -39., -29., -19.,  -9.,   1.,
        11.,  21.,  31.,  41.,  51.,  61.,  71.,  81.,  91.]), 'gamma': [0.0001], 'epsilon': [0.01]}
best  {'C': 1.0, 'epsilon': 0.01, 'gamma': 0.0001, 'kernel': 'rb